# Imports

In [1]:
import os
import logging
import json

import optuna

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from kaggle.api.kaggle_api_extended import KaggleApi

from utils import DepressionDataProcessor

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

from catboost import CatBoostClassifier

plt.style.use('ggplot')
%matplotlib inline

ISKAGGLE = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "") != ""

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


# Data Loading

In [2]:

processor = DepressionDataProcessor("anthonytherrien/depression-dataset")
processor.download_and_extract()

2025-02-21 09:27:53,366 - INFO - Downloading dataset using Kaggle API...


datasets\depression-dataset
Dataset URL: https://www.kaggle.com/datasets/anthonytherrien/depression-dataset


2025-02-21 09:27:55,932 - INFO - Dataset downloaded and extracted successfully.


In [3]:
raw_df = processor.load_data("depression_data.csv")

In [4]:

processed_df = processor.preprocess(raw_df)
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413768 entries, 0 to 413767
Data columns (total 20 columns):
 #   Column                           Non-Null Count   Dtype   
---  ------                           --------------   -----   
 0   Age                              413768 non-null  int64   
 1   Marital Status                   413768 non-null  category
 2   Education Level                  413768 non-null  category
 3   Number of Children               413768 non-null  category
 4   Smoking Status                   413768 non-null  category
 5   Physical Activity Level          413768 non-null  category
 6   Employment Status                413768 non-null  category
 7   Income                           413768 non-null  float64 
 8   Alcohol Consumption              413768 non-null  category
 9   Dietary Habits                   413768 non-null  category
 10  Sleep Patterns                   413768 non-null  category
 11  History of Mental Illness        413768 non-null  ca

In [5]:
for feature in processed_df.select_dtypes(include='category').columns:
        print(f'{feature}: {processed_df[feature].unique()}')

Marital Status: ['Married', 'Widowed', 'Divorced', 'Single']
Categories (4, object): ['Divorced', 'Married', 'Single', 'Widowed']
Education Level: ['Bachelor's Degree', 'High School', 'Master's Degree', 'Associate Degree', 'PhD']
Categories (5, object): ['Associate Degree', 'Bachelor's Degree', 'High School', 'Master's Degree', 'PhD']
Number of Children: ['2', '1', '3', '0', '4']
Categories (5, object): ['0', '1', '2', '3', '4']
Smoking Status: ['Non-smoker', 'Former', 'Current']
Categories (3, object): ['Current', 'Former', 'Non-smoker']
Physical Activity Level: ['Active', 'Sedentary', 'Moderate']
Categories (3, object): ['Active', 'Moderate', 'Sedentary']
Employment Status: ['Unemployed', 'Employed']
Categories (2, object): ['Employed', 'Unemployed']
Alcohol Consumption: ['Moderate', 'High', 'Low']
Categories (3, object): ['High', 'Low', 'Moderate']
Dietary Habits: ['Moderate', 'Unhealthy', 'Healthy']
Categories (3, object): ['Healthy', 'Moderate', 'Unhealthy']
Sleep Patterns: ['Fair

In [6]:
features = [
    'Income Bucket Employment Status',
    'Marital Status', 
    'Number of Children', 
    'Smoking Status',
    'Physical Activity Level', 
    'Alcohol Consumption',
    'Dietary Habits', 
    'Sleep Patterns',
    'History of Substance Abuse', 
    'Family History of Depression',
    'Chronic Medical Conditions', 
    'Age Bucket', 
    'Attended University',
    'Education Level',
]
target = 'History of Mental Illness'

# Prepare the data
X = processed_df[features]
y = processed_df[target]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Model Hyper Parameter Tuning

In [7]:
categorical_features_indices = [X_train.columns.get_loc(col) for col in X_train.select_dtypes(include='category').columns]

X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

class_weights = y_train_sub.value_counts(normalize=True).to_dict()
class_weights = class_weights[0] / class_weights[1]

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'scale_pos_weight': class_weights,
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'task_type': "GPU",
        'devices': '0',
        'eval_metric': 'Recall',
        'verbose': False,
        'scale_pos_weight': class_weights
    }

    model = CatBoostClassifier(**params)
    model.fit(X_train_sub, y_train_sub, cat_features=categorical_features_indices, eval_set=(X_valid, y_valid), early_stopping_rounds=50)

    y_pred = model.predict(X_valid)
    return recall_score(y_valid, y_pred, average='weighted')

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

best_params = study.best_params
best_params.update({'task_type': "GPU", 'devices': '0', 'eval_metric': 'Recall', 'verbose': False})

[I 2025-02-21 09:27:57,537] A new study created in memory with name: no-name-92cbdeac-c6dd-4ef2-bb5b-eb202cc2e96f
[I 2025-02-21 09:27:59,697] Trial 0 finished with value: 0.6309834901741613 and parameters: {'iterations': 676, 'depth': 4, 'learning_rate': 0.009025066763783507, 'bagging_temperature': 0.7424545165475553, 'l2_leaf_reg': 1.8616708110406823, 'border_count': 209}. Best is trial 0 with value: 0.6309834901741613.
[I 2025-02-21 09:28:02,177] Trial 1 finished with value: 0.6088092684621543 and parameters: {'iterations': 513, 'depth': 7, 'learning_rate': 0.029180745739857926, 'bagging_temperature': 0.9448287099678437, 'l2_leaf_reg': 4.768622348217815, 'border_count': 150}. Best is trial 0 with value: 0.6309834901741613.
[I 2025-02-21 09:28:04,167] Trial 2 finished with value: 0.6150325513949519 and parameters: {'iterations': 993, 'depth': 5, 'learning_rate': 0.003505363418178011, 'bagging_temperature': 0.024399323990328847, 'l2_leaf_reg': 5.6809319313852065, 'border_count': 250}. 

In [8]:
with open('catboost_best_params.json', 'w') as f:
    json.dump(best_params, f)